In [295]:
import pandas as pd
import numpy as np

df = pd.read_csv('../database/stock_price/stock_price.csv',
                 dtype=dict(Price=np.int32, Volume=np.int32, StockCode=np.int32),
                 parse_dates=['Date'])


def smoothing(stock_df, stock_code, window_size):
    data = stock_df[stock_df['StockCode'] == stock_code].sort_values('Date').set_index('Date')['Price']
    if len(data) % window_size == 0:
        sec_num = len(data) // window_size
    else:
        sec_num = len(data) // window_size + 1
    output = pd.Series(list(data[window_size * i:window_size * (i + 1)].median() for i in range(sec_num)))
    return output

def train_test_split(smooth_sris, num_feature):
    train, test = pd.DataFrame(), pd.DataFrame()

    for i in range(num_feature + 1):
        train = pd.concat([train, smooth_sris.shift(-(i))], axis=1, ignore_index=True)
    for j in range(num_feature):
        test = pd.concat([test, smooth_sris.shift(-(j))], axis=1, ignore_index=True)

    X_train = train.dropna().iloc[:, :num_feature].values
    y_train = train.dropna().iloc[:, num_feature].values
    X_test = test.dropna().iloc[-1].values    
    return X_train, y_train, X_test

test = smoothing(stock_df=df, stock_code=11930, window_size=1)
interval_size = 20
X_train, y_train, X_test = train_test_split(test, interval_size)

In [296]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test.reshape(1,X_test.shape[0]))

In [297]:
import tensorflow as tf

In [298]:
def GRU():
    tf.random.set_seed(20190530)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.GRU(units=128, return_sequences=True, input_shape=(interval_size,1)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.GRU(units=128, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.GRU(units=128, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.GRU(units=128, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.GRU(units=128))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    return model

def LSTM():
    tf.random.set_seed(20190530)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True, input_shape=(interval_size,1)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.LSTM(units=128, return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.LSTM(units=128))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Dense(units=1))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    return model

In [299]:
a = X_train_sc.reshape(X_train.shape[0], interval_size, 1)
b = y_train.reshape(y_train.shape[0],1)
c = X_test_sc.reshape(1, interval_size, 1)

In [300]:
gru = GRU(); lstm = LSTM()
gru.fit(x=a, y=b, batch_size=50, epochs=100, verbose=0)
lstm.fit(x=a, y=b, batch_size=50, epochs=100, verbose=0)

In [301]:
(gru.predict(c) + lstm.predict(c))/2

array([[1338.4404]], dtype=float32)

In [303]:
gru.predict(c)

array([[1721.5077]], dtype=float32)

In [304]:
 lstm.predict(c)

array([[955.3732]], dtype=float32)